# OT-2: Functional System Sizing

In [5]:
import handcalcs.render
from turborocket.fluids.fluids import IncompressibleFluid
from turborocket.transient.start_up import MainEngine
import numpy as np
from turborocket.sizing.pump import Barske, DiffuserType
import CoolProp.CoolProp as CP

> Date: 30/11/2025

> Author: Elias Aoubala

## 1 - Background

This document contains the high level system sizing of the OT-2 turbopump, intended for use in the R2S 2026 competition. The high level functional specifications for the turbopump to be developed is performed here-in.

This document is split into the following sections:

- Section 1: Background
- Section 2: Derivation of Driving Requirements
  - Here, we describe the method for what drives the high level sizing of the turbo-pump.
- Section 3: Power Budget Break-down
  - Based on the derivation of the driving requirements, we derive the individual power budet assesments of the system
  - High level assumptions and estimations are taken on the power losses due to bearings and what not.
- Section 4: Shaft Speed Sensitvity Study and Selection
  - A sensitivity study is performed to derive the shaft speed of the system.
  - The impact of shaft speed on the expected shaft stresses, component life times and overall system packaging is performed.
- Section 5: Impeller Design Trade-off
  - Based on the shaft speed selections, we will then conduct a study on the impeller design.
  - A trade-off of the traditional "Barske" style impeller compared to other alternative solutions is performed - with a final recommendation made at the end.
- Section 6: Turbine Design Trade-off
  - Similar to the impeller tade study, a trade study is performed on the alternative options for the turbine side of the turbopump.
  - The trade study focuses to compare the axial based turbine design, as used previously - to that of a radial pump, and evaluate which would be the best for the present requirements.
- Section 7: Final Turbopump Design Specification
  - Based on the trade studies performed, this section presents the final high level functional specification fo the OT-2 turbopump assembly.

## 2 - Derivation of Driving Requirements

The approach used for the sizing of the OT-2 turbopump was based on the functional requiements for a future development engine by the group called "Athena".

### 2.1 - Athena: A High Thrust-Class Engine

The *Athena* Engine is a theoretical 20 kN engine used at present as a place-holder for the sizing of the OT-2 turbopump. It is a Nitrous/IPA Turbopump driven engine assembly, with the IPA being pumped by the `Man-Eay and The Dirty Bubble (OT-2)` Turbopump stack.

The main purpose for picking such a high-thrust chamber for driving the sizing of the turbopump is fundementally two fold:

- **Turbopump Size Does Not Scale Small**:
  -  does not scale with the capacity at the low-end - as you reach minimum dimensions and spacings.
- **Off-nominal performance is not an issue**:
  - For Barske and supersonic turbines, off-nominal performance is generaly not an issue due to unique incidence and flat head curves, such that operating at lower mass flow rates, the head and fludic performance is comparable
  - Caveat: The "effective" efficiency is worse as less propellant is being pumped, but in practice this doesnt matter.
- **Growth Potential**
  - Currently, there is no "Athena" class engine build by Amateurs - however sizing the OT-2 turbopump ahead of time allows for growth in the developed engines by Amateurs.


The sizing approach for this engine uses the `MainEngine` object of the `transient` class set within the in-house developed `turborocket` python package.

For the combustion modelling, we will use a reduced chemical mechanism file with the `cantera` python package - which is has been embeded within the `turborocket` package. This mechanism is heritage from the previously developed OT-1 `Mermamid Man and Barnacle Boy` turbopump.

#### 2.1.1 - High Level Inputs

Inorder for the `turborocket` software to design the engine, you need to define the key propellants, nominal combustion chamber and the nominal mixture ratio of the engine.

Below, these high level parameters are listed.

In [6]:
%%render params sci_not

P_c = 25e5
Ox = "N2O"
Fu = "C3H8O,2propanol"
MR = 4

<IPython.core.display.Latex object>

N.B: The reduced mechanism file used for the `cantera` based calculations orginate from the already available `nasa_gas.yaml` reactants list. This was reduced to only include molecules relevant to the reaction, while ensuring the following key thermodynamic properties were maintained:

- Specific Heat Ratio ($\gamma$)
- Specific Heat Capacity ($C_p$)
- Adiabatic Flame Temperature ($T_o$)

As the original mechanism file originates from the `nasa_gas.yaml` file, we must refer to Isopropyl Alcohol by its molecular formulation. It should be noted that transient combustion has not been considered in the present analysis - with only steady analysis used.

An additional parameter we will assume is the C* efficiency of the engine. For this, we refer to similar engines that have fired previously, and take a conservative margin.

In [15]:
%%render

eta_c = 0.90 # \%

<IPython.core.display.Latex object>

#### 2.1.2 - Injector Parameters

For the sizing of the engine, we need to define what the injector inlet parameters will be.

This is expected to be iterative - but at this point, we can arbitrarily select these. In addition, we must define the generic properties for the propellants to allow for the sizing of the oxidiser and fuel side injector.

We will model both propellants as perfect incompressible fluids for now to keep things simple, and assume a reduced discharge-coefficient on the nitrous side which is experimentally based. 

In [7]:
%%render params sci_not

P_ipa = 50e5 # Pa
P_n2o = 50e5 # Pa

rho_ipa = 786 # kg/m\^3
rho_n2o = CP.PropsSI("D", "P", P_n2o, "Q", 0, "NitrousOxide") # kg/m\^3

Cd_f = 0.8
Cd_o = 0.3

<IPython.core.display.Latex object>

From these parameters, we can define our incompressible fluid objects.

In [8]:
ATHENA_IPA = IncompressibleFluid(rho=rho_ipa, P=P_ipa)

ATHENA_N2O = IncompressibleFluid(rho=rho_n2o, P=P_n2o)

#### 2.1.2 - Main Engine Instantiation

Based on these inputs, we can instantiate our combustion chamber object.

In [19]:
ATHENA = MainEngine(
    Ox=Ox, Fu=Fu, Pcc=P_c, MR=MR
)

We need to now inform turborocket what combustion solver it will use for the combustion calculations. In this case, we will use the aforementioned reduced mechanism file `n2o_ipa.yaml`.

In [20]:
ATHENA.comb_object(combustion_file="n2o_ipa.yaml")

No look-up table will be used at this time, as we dont intend to do iterative combustion evaluations here - this is only relevant for transient or comparable simulations.

Now we can instantiate our injector geometry

In [21]:
ATHENA.injector_cond(ox_in=ATHENA_N2O, fu_in=ATHENA_IPA, cdo=Cd_o, cdf=Cd_f)

We can finally size the system.

#### 2.1.3 - Engine Sizing

For the engine sizing, we will build a basic iterative manual loop, where the user will iterate through different mass flow rates until a target thrust is met.

**Target Thrust:**

In [22]:
%%render parmas sci_not

F_t = 25e3 # N

<IPython.core.display.Latex object>

**Initial Input Parameter**

In [23]:
%%render params

m_dot_t = 1.5 # Total - kg/s

<IPython.core.display.Latex object>

**Loop Parameters (relaxation and error)**

In [24]:
error = 0
relax = 0.4

**Integration loop** - keep running cell till error converges

In [58]:
m_dot_t -= error*m_dot_t*relax

F_a = ATHENA.size_system(m_dot=m_dot_t, eta_c = eta_c)["F"]

error = (F_a - F_t)/ F_t

print(f"Relative Error: {error*100:.2f} %")
print(f"Mass Flow (kg/s): {m_dot_t}")

Relative Error: -0.00 %
Mass Flow (kg/s): 12.393232315651359


**Final Mass Flow**

In [60]:
%%render param 2

m_dot_t = m_dot_t # kg/s

mdot_ox = ATHENA.size_system(m_dot=m_dot_t, eta_c = eta_c)["m_dot_o"] # kg/s

mdot_f = ATHENA.size_system(m_dot=m_dot_t, eta_c = eta_c)["m_dot_f"] # kg/s

<IPython.core.display.Latex object>

## 3 - Power Budget Breakdown

For the power budget breakdown, we will at this stage assume some losses and efficiencies for the sizing of the system.

### 3.1 - Fluidic Power Requirements

#### 3.1.1 - Pump Power

In [2]:
pump_dic = {"dp": 35e5, 
            "m_dot": 2, 
            "N": 30e3 * (2 * np.pi) / 60}

In [3]:
IPA = IncompressibleFluid(rho=787, P=15e5, mue=2.37e-3)

The following geometric parameters and performance assumptions have been used:

In [4]:
pump_geo = {
    "fluid": IPA,
    "l_1": 10e-3,
    "l_2": 6e-3,
    "v_0": 3.5,
    "v_3f": 0.61,
    "d_1f": 0.9,
    "a_3f": 3.5,
    "delta_div": 8,
    "diffuser_type": DiffuserType.rectangular,
    "psi": 0.5,
}

We can also evaluate for our specific speed of the pump to compare it to others of similar geometry

In [5]:
M3S_TO_GPM = 15850.323140625
g = 9.81
M_TO_FT = 3.28084

In [6]:
%%render

Q = (pump_dic["m_dot"]/pump_geo["fluid"].get_density()) * M3S_TO_GPM # gpm

H = pump_dic["dp"]/(pump_geo["fluid"].get_density() * g) * M_TO_FT # ft

N = pump_dic["N"] * 60/(2*np.pi) # rpm

N_s = (N * (Q) ** (1 / 2)) / H ** (0.75)

<IPython.core.display.Latex object>

Below a list of the key geometric parameters for the pump has been derived.

In [7]:
pump = Barske(**pump_dic)

pump.size_pump(**pump_geo)

,Eye Diameter - d_0 (mm),Inlet Diameter - d_1 (mm),Exit Diameter - d_2 (mm),Entrance Axial Blade Length - l_1 (mm),Exit Axial Blade Legnth - l_2 (mm),Axial Clearance - c_1 (mm),Radial Clearance - c_2 (mm),Diffuser Type,Diffuser Throat - d_3 (mm),Diffuser Exit - d_4 (mm),Diffuser Length - L (mm),Inlet Eye Velocity - v_o (m/s),Impeller Inlet Velocity - v_1 (m/s),Relative Exit Velocity - w_2 (m/s)
0,30.405256,27.364731,53.874225,10.0,6.0,0.538742,2.693711,DiffuserType.rectangular,8.204899,28.717148,145.952233,3.5,2.956067,2.502494


Finally, this is the expected performance metrics of the pump

In [8]:
performance = pump.get_pump_performance(fluid=pump_geo["fluid"], m_dot=pump_dic["m_dot"], psi=pump_geo["psi"], N=pump_dic["N"])

performance

,Head Rise (m),Head Rise (Bar),Efficiency (%),Hydraulic Power (kW),Required Shaft Power (kW)
0,453.340276,35.0,58.80139,8.894536,15.126405


#### 3.1.2 - Expeller

For the expeller, we will size an equivalent pump that means the head pressure plus an additional 5 Bar.

In [9]:
expeller_dic = {"dp": IPA.get_pressure() + 5e5,
                "m_dot": 2,
                "N": 30e3 * (2 * np.pi) / 60}

In [10]:
IPA = IncompressibleFluid(rho=787, P=15e5, mue=2.37e-3)

The following geometric parameters and performance assumptions have been used:

In [15]:
expeller_geo = {
    "fluid": IPA,
    "d_1": 25e-3,
    "l_1": 10e-3,
    "l_2": 10e-3,
    "psi": 0.2,
}

We can also evaluate for our specific speed of the pump to compare it to others of similar geometry

In [16]:
expeller = Barske(**expeller_dic)

expeller.size_expeller(**expeller_geo)

,Inlet Diameter - d_1 (mm),Exit Diameter - d_2 (mm),Entrance Axial Blade Length - l_1 (mm),Exit Axial Blade Legnth - l_2 (mm),Axial Clearance - c_1 (mm),Radial Clearance - c_2 (mm)
0,25.0,47.301313,10.0,10.0,0.473013,2.365066


Finally, this is the expected performance metrics of the pump

In [18]:
Pw = expeller.get_pump_paddle_power(fluid = IPA,
                                    N = expeller_dic["N"])

print(f"Expeller Power: {Pw/1e3:.2f} kW")

Expeller Power: 1.54 kW


### 3.2 - Mechanical Losses

The key mechanical losses for the system are for the bearings. In this case, we will assume a similar performance to last year

In [21]:
%%render

P_bearing = 400 # W

N_bearing = 3

P_bearing_2 = P_bearing * N_bearing # W

<IPython.core.display.Latex object>

### 3.3 - Total Power Requirement

Based on the bearing and expeller requirements, the following total power requirement is the following:

In [34]:
P_total = Pw + performance["Required Shaft Power (kW)"][0] * 1e3 + P_bearing_2

print(f"Total Power: {P_total/1e3:.2f} kW")

Total Power: 17.86 kW
